In [ ]:
import pandas as pd
from torch.utils.data import Dataset
import torch
rnn_encoder, rnn_decoder, transformer_encoder, transformer_decoder = None, None, None, None
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if __name__=='__main__':
    print('Using device:', DEVICE)

In [ ]:
# if __name__ == '__main__':
#     os.system("wget http://www.manythings.org/anki/spa-eng.zip")
#     import zipfile
#     with zipfile.ZipFile('spa-eng.zip', 'r') as zip_ref:
#         zip_ref.extractall('data')

In [ ]:
if __name__ == '__main__':

    total_num_examples = 50000
    dat = pd.read_csv("data/spa.txt",
                    sep="\t",
                    header=None,
                    usecols=[0,1],
                    names=['eng', 'es'],
                    nrows=total_num_examples,
                    encoding="UTF-8"
    ).sample(frac=1).reset_index().drop(['index'], axis=1)

    print(dat) # Visualize the data

In [ ]:
from src.process.helpers import preprocess_sentence 

In [ ]:
if __name__ == '__main__':
    data = dat.copy()
    data['eng'] = dat.eng.apply(lambda w: preprocess_sentence(w))
    data['es'] = dat.es.apply(lambda w: preprocess_sentence(w))
    print(data) # Visualizing the data

In [ ]:
import numpy as np
import random
from torch.utils.data import DataLoader

In [ ]:
if __name__ == '__main__':
    # HYPERPARAMETERS (You may change these if you want, though you shouldn't need to)
    BATCH_SIZE = 64
    EMBEDDING_DIM = 256

In [ ]:
from src.process.helpers import build_vocabulary

In [ ]:
if __name__ == '__main__':
    src_vocab_list = build_vocabulary(data['es'])
    trg_vocab_list = build_vocabulary(data['eng'])

In [ ]:
from src.process.objects.vocabData import Vocab_Lang, MyData
from src.process.helpers import preprocess_data_to_tensor, train_test_split

In [ ]:
if __name__ == '__main__':
    src_vocab = Vocab_Lang(src_vocab_list)
    trg_vocab = Vocab_Lang(trg_vocab_list)

    src_tensor, trg_tensor, max_length_src, max_length_trg = preprocess_data_to_tensor(data, src_vocab, trg_vocab)
    src_tensor_train, src_tensor_val, trg_tensor_train, trg_tensor_val = train_test_split(src_tensor, trg_tensor)

    # Create train and val datasets
    train_dataset = MyData(src_tensor_train, trg_tensor_train)
    train_dataset = DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)

    test_dataset = MyData(src_tensor_val, trg_tensor_val)
    test_dataset = DataLoader(test_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)

In [ ]:
if __name__ == '__main__':
    idxes = random.choices(range(len(train_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    print('Source:', src)
    print('Source Dimensions: ', src.size())
    print('Target:', trg)
    print('Target Dimensions: ', trg.size())

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import time
from tqdm.notebook import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction, corpus_bleu

In [ ]:
from src.evaluation.sanityChecks import sanityCheckModel
from src.models.rnn.encoder import RnnEncoder

In [ ]:
if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    embedding_dim = [2, 5, 8]
    hidden_units = [50, 100, 200]
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for hu in hidden_units:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['hidden_units'] = hu
            inputs.append(inp)
    # Test init
    expected_outputs = [8110, 31210, 122410, 8575, 32125, 124225, 9040, 33040, 126040]

    sanityCheckModel(inputs, RnnEncoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    batch_sizes = [1, 2]
    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            inp['embedding_dim'] = EMBEDDING_DIM
            inp['src_vocab'] = sanity_vocab
            inp["batch_size"] = b
            inp['hidden_units'] = hu
            inputs.append(inp)
    expected_outputs = [torch.Size([16, 1, 50]), torch.Size([16, 2, 50]), torch.Size([16, 1, 100]), torch.Size([16, 2, 100]), torch.Size([16, 1, 200]), torch.Size([16, 2, 200])]

    sanityCheckModel(inputs, RnnEncoder, expected_outputs, "forward")

In [ ]:
from src.evaluation.sanityChecks import sanityCheckDecoderModelForward
from src.models.rnn.decoder import RnnDecoder
import torch.nn.functional as F

In [ ]:
if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    embedding_dim = [2, 5, 8]
    hidden_units = [50, 100, 200]
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for hu in hidden_units:
            inp = {}
            inp['trg_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['hidden_units'] = hu
            inputs.append(inp)
    # Test init
    expected_outputs = [21016, 82016, 324016, 21481, 82931, 325831, 21946, 83846, 327646]
    sanityCheckModel(inputs, RnnDecoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    hidden_units = [50, 100, 200]
    batch_sizes = [1, 2, 4]
    embedding_dims = iter([50,80,100,120,150,200,300,400,500])
    encoder_outputs = iter([torch.rand([16, 1, 50]), torch.rand([16, 2, 50]), torch.rand([16, 4, 50]), torch.rand([16, 1, 100]), torch.rand([16, 2, 100]), torch.rand([16, 4, 100]), torch.rand([16, 1, 200]), torch.rand([16, 2, 200]),torch.rand([16, 4, 200])])
    expected_fc_outs = [torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5]),torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5]),torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5])]
    expected_dec_hs = [torch.Size([1, 1, 50]), torch.Size([1, 2, 50]), torch.Size([1, 4, 50]), torch.Size([1, 1, 100]), torch.Size([1, 2, 100]), torch.Size([1, 4, 100]), torch.Size([1, 1, 200]), torch.Size([1, 2, 200]), torch.Size([1, 4, 200])]
    expected_attention_weights = [torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1]), torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1]), torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1])]
    expected_outputs = (expected_fc_outs, expected_dec_hs, expected_attention_weights)

    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            edim = next(embedding_dims)
            inp['embedding_dim'] = edim
            inp['trg_vocab'] = sanity_vocab
            inp["batch_size"] = b
            inp['hidden_units'] = hu
            inp['encoder_outputs'] = next(encoder_outputs)
            inputs.append(inp)

    sanityCheckDecoderModelForward(inputs, RnnDecoder, expected_outputs)

In [ ]:
from src.models.rnn.encoder import RnnEncoder

In [ ]:
if __name__ == '__main__':
    # HYPERPARAMETERS - feel free to change
    LEARNING_RATE = 0.001
    HIDDEN_UNITS=256
    N_EPOCHS=10

    rnn_encoder = RnnEncoder(src_vocab, EMBEDDING_DIM, HIDDEN_UNITS).to(DEVICE)
    rnn_decoder = RnnDecoder(trg_vocab, EMBEDDING_DIM, HIDDEN_UNITS).to(DEVICE)

    rnn_model_params = list(rnn_encoder.parameters()) + list(rnn_decoder.parameters())
    optimizer = torch.optim.Adam(rnn_model_params, lr=LEARNING_RATE)

    print('Encoder and Decoder models initialized!')

In [ ]:
from src.models.rnn.train import train_rnn_model

In [ ]:
if __name__ == '__main__':
    train_rnn_model(rnn_encoder, rnn_decoder, train_dataset, optimizer, trg_vocab, DEVICE, N_EPOCHS)

In [ ]:
from src.models.rnn.inference import decode_rnn_model

In [ ]:
if __name__ == '__main__':
    rnn_encoder.eval()
    rnn_decoder.eval()
    idxes = random.choices(range(len(test_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    curr_output, _ = decode_rnn_model(rnn_encoder, rnn_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), DEVICE)
    for i in range(len(src)):
        print("Source sentence:", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[i]] if x != '<pad>']))
        print("Target sentence:", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[i]] if x != '<pad>']))
        print("Predicted sentence:", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in curr_output[i]] if x != '<pad>']))
        print("----------------")

In [ ]:
from src.models.rnn.inference import evaluate_rnn_model

In [ ]:
if __name__ == '__main__':
    rnn_save_candidate, rnn_scores = evaluate_rnn_model(rnn_encoder, rnn_decoder, test_dataset, trg_tensor_val, DEVICE)

In [ ]:
from src.models.transformer.encoder import TransformerEncoder

In [ ]:
if __name__=="__main__":
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    dimf = [50, 100, 150]
    embedding_dim = [4, 8, 12]
    max_len = 16
    num_layers = iter([1,1,1,2,2,2,3,3,3])
    nheads = iter([1, 1, 1, 1, 2, 2, 2, 4, 4])
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for df in dimf:
        for ed in embedding_dim:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['num_heads'] = next(nheads)
            inp['dim_feedforward'] = df
            inp['num_layers'] = next(num_layers)
            inp['max_len_src'] = max_len
            inp['device'] = DEVICE
            inputs.append(inp)
    # Test init
    expected_outputs = [570, 1218, 1994, 2020, 4096, 6428, 4370, 8674, 13362]

    sanityCheckModel(inputs, TransformerEncoder, expected_outputs, "init")

In [ ]:
if __name__=="__main__":
    # Set random seed
    torch.manual_seed(42)
    # Test forward
    inputs = []
    embedding_dims = [32,64,128]
    batch_sizes = [1, 2]
    dimf = 100
    nheads = iter([1,1,2,2,4,4])
    num_layers = iter([1,1,2,2,3,3])
    max_len = 16
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    for ed in embedding_dims:
        for b in batch_sizes:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['num_heads'] = next(nheads)
            inp['dim_feedforward'] = dimf
            inp['num_layers'] = next(num_layers)
            inp['max_len_src'] = max_len
            inp['device'] = DEVICE
            inp["batch_size"] = b
            inputs.append(inp)
    expected_outputs = [torch.Size([16, 1, 32]), torch.Size([16, 2, 32]), torch.Size([16, 1, 64]), torch.Size([16, 2, 64]), torch.Size([16, 1, 128]), torch.Size([16, 2, 128])]
    # expected_outputs.to(DEVICE)

    sanityCheckModel(inputs, TransformerEncoder, expected_outputs, "forward")

In [ ]:
from src.models.transformer.decoder import TransformerDecoder
from src.evaluation.sanityChecks import sanityCheckTransformerDecoderModelForward

In [ ]:
if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    hidden_units = [50, 100, 200]
    embedding_dim = [8, 16]
    num_heads = [1, 2]
    dim_feedforward = [50, 100]
    num_layers = [1, 2]
    max_lens = 64
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for df in dim_feedforward:
            for nh in num_heads:
                for nl in num_layers:
                    inp = {}
                    inp['trg_vocab'] = sanity_vocab
                    inp['embedding_dim'] = ed
                    inp['num_heads'] = nh
                    inp['num_layers'] = nl
                    inp['dim_feedforward'] = df
                    inp['max_len_trg'] = max_lens
                    inp['device'] = DEVICE
                    inputs.append(inp)
    # Test init
    expected_outputs = [1567, 3049, 1567, 3049, 2417, 4749, 2417, 4749]
    sanityCheckModel(inputs, TransformerDecoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    batch_sizes = [1, 2, 4]
    num_heads = 2
    num_layers = 1
    embedding_dims = iter([100, 100, 200, 200, 200, 400, 400, 800, 800])
    max_lens = iter([16, 16, 16, 32, 32, 32, 64, 64, 128])
    expected_outputs = [torch.Size([16, 1, 5]),torch.Size([16, 2, 5]),torch.Size([16, 4, 5]),torch.Size([32, 1, 5]),torch.Size([32, 2, 5]),torch.Size([32, 4, 5]),torch.Size([64, 1, 5]),torch.Size([64, 2, 5]),torch.Size([128, 4, 5])]

    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            edim = next(embedding_dims)
            inp['embedding_dim'] = edim
            inp['trg_vocab'] = sanity_vocab
            inp['num_heads'] = num_heads
            inp['num_layers'] = num_layers
            inp["batch_size"] = b
            inp['dim_feedforward'] = hu
            inp['max_len_trg'] = next(max_lens)
            inp['device'] = DEVICE
            inputs.append(inp)

    sanityCheckTransformerDecoderModelForward(inputs, TransformerDecoder, expected_outputs)

In [ ]:
from src.models.transformer.train import train_transformer_model

In [ ]:
if __name__ == '__main__':
    # HYPERPARAMETERS - feel free to change
    LEARNING_RATE = 0.001
    DIM_FEEDFORWARD=512
    N_EPOCHS=10
    N_HEADS=2
    N_LAYERS=2
    DROPOUT=0.1

    transformer_encoder = TransformerEncoder(src_vocab, EMBEDDING_DIM, N_HEADS,
                                 N_LAYERS,DIM_FEEDFORWARD,
                                 max_length_src, DEVICE, DROPOUT).to(DEVICE)
    transformer_decoder = TransformerDecoder(trg_vocab, EMBEDDING_DIM, N_HEADS,
                              N_LAYERS,DIM_FEEDFORWARD,
                              max_length_trg, DEVICE, DROPOUT).to(DEVICE)

    transformer_model_params = list(transformer_encoder.parameters()) + list(transformer_decoder.parameters())
    optimizer = torch.optim.Adam(transformer_model_params, lr=LEARNING_RATE)

    print('Encoder and Decoder models initialized!')

In [ ]:
if __name__ == '__main__':
    train_transformer_model(train_dataset, transformer_encoder, transformer_decoder, optimizer, DEVICE, N_EPOCHS)

In [ ]:
from src.models.transformer.inference import decode_transformer_model

In [ ]:
if __name__ == '__main__':
    transformer_encoder.eval()
    transformer_decoder.eval()
    idxes = random.choices(range(len(test_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    curr_output, _, _ = decode_transformer_model(transformer_encoder, transformer_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), DEVICE)
    for i in range(len(src)):
        print("Source sentence:", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[i]] if x != '<pad>']))
        print("Target sentence:", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[i]] if x != '<pad>']))
        print("Predicted sentence:", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in curr_output[i]] if x != '<pad>']))
        print("----------------")

In [ ]:
from src.models.transformer.inference import evaluate_transformer_model

In [ ]:
if __name__ == '__main__':
    transformer_save_candidate, transformer_scores = evaluate_transformer_model(transformer_encoder, transformer_decoder, test_dataset, trg_tensor_val, DEVICE)

In [ ]:
if __name__=='__main__':
    print()
    if rnn_encoder is not None and rnn_decoder is not None:
        print("Saving RNN model...")
        torch.save(rnn_encoder, 'models/rnn_encoder.pt')
        torch.save(rnn_decoder, 'models/rnn_decoder.pt')
        print("RNN model saved successfully.\n")
    if transformer_encoder is not None and transformer_decoder is not None:
        print("Saving Transformer model...")
        torch.save(transformer_encoder, 'models/transformer_encoder.pt')
        torch.save(transformer_decoder, 'models/transformer_decoder.pt')
        print("Transformer model saved successfully.\n")